In [ ]:
!pip install pandas lxml pandas_datareader setuptools plotly yfinance

Question 1: [Index] S&P 500 Stocks Added to the Index
Which year had the highest number of additions?

Using the list of S&P 500 companies from Wikipedia's S&P 500 companies page, download the data including the year each company was added to the index.

Hint: you can use pandas.read_html to scrape the data into a DataFrame.

Steps:

Create a DataFrame with company tickers, names, and the year they were added.
Extract the year from the addition date and calculate the number of stocks added each year.
Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).
Context:

"Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 (Nasdaq article).

Additional: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


In [ ]:
import pandas as pd

# Scrape the S&P 500 companies table from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]  # The first table contains the list of S&P 500 companies

# Display the first few rows
sp500_df.head()


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
sp500_df['Year added'] = pd.to_datetime(sp500_df['Date added'], errors='coerce').dt.year

In [21]:
#quick check 
year_added = sp500_df['Year added']
valid_years = year_added.dropna()
valid_years = valid_years[(valid_years >= 1900) & (valid_years <= 2025)]
if len(valid_years) == len(sp500_df):
    print("data valid")
else:
    print("check your data")


data valid


In [66]:
# Count the number of companies added each year (excluding 1957)
year_counts = sp500_df[sp500_df['Year added'] != 1957]['Year added'].value_counts()

# Get the top years with the most additions
year_counts.nlargest(1)
print("Q: Year had the highest number of addition\nA:", year_counts.nlargest(1).index[0])


Q: Year had the highest number of addition
A: 2017


In [71]:
#add the number of years
sp500_df['Years in sp500'] = abs((datetime.today() - pd.to_datetime(sp500_df['Date added'], errors='coerce')).dt.days) // 365

# Count the number of companies been index for more than 20 years
more_than_20_years = sp500_df[sp500_df['Years in sp500'] > 20]
print("Q: How many current S&P 500 stocks have been in the index for more than 20 years?\nA:", len(more_than_20_years))

Q: How many current S&P 500 stocks have been in the index for more than 20 years?
A: 215


Question 2. [Macro] Indexes YTD (as of 1 May 2025)
How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:

United States - S&P 500 (^GSPC)
China - Shanghai Composite (000001.SS)
Hong Kong - HANG SENG INDEX (^HSI)
Australia - S&P/ASX 200 (^AXJO)
India - Nifty 50 (^NSEI)
Canada - S&P/TSX Composite (^GSPTSE)
Germany - DAX (^GDAXI)
United Kingdom - FTSE 100 (^FTSE)
Japan - Nikkei 225 (^N225)
Mexico - IPC Mexico (^MXX)
Brazil - Ibovespa (^BVSP)
Hint: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context:

Global Valuations: Who's Cheap, Who's Not? article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

Additional: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend? Note: For simplicity, ignore currency conversion effects.)

In [100]:
from datetime import date, timedelta

def get_report_by_date(data,ticker_to_name, start=(date.today() - timedelta(days=1)).strftime("%Y-%m-%d"), end=date.today().strftime("%Y-%m-%d")):
    #set the start date jan
    print(f'Period for indexes: {start} to {end} ')
    data_prepped = data[(data.index >= start) & (data.index <= end)]

    # Rename columns in data for easier reading
    data_renamed = data_prepped.rename(columns=ticker_to_name)

    #data_renamed.head()

    # Calculate YTD return for each index
    ytd_returns = (data_renamed.iloc[-1] - data_renamed.iloc[0]) / data_renamed.iloc[0] * 100

    # Sort by return descending
    ytd_returns_sorted = ytd_returns.sort_values(ascending=False)
    ytd_returns_ranked = ytd_returns_sorted.reset_index()
    ytd_returns_ranked.columns = ['Index', 'YTD Return (%)']
    ytd_returns_ranked['Rank'] = ytd_returns_ranked.index + 1
    ytd_returns_ranked = ytd_returns_ranked[['Rank', 'Index', 'YTD Return (%)']]
    ytd_returns_ranked['YTD Return (%)'] = ytd_returns_ranked['YTD Return (%)'].map('{:.2f}%'.format)
    print(ytd_returns_ranked.to_string(index=False))

In [ ]:
#get data from yahoo finance
index_tickers = {
    "United States - S&P 500": "^GSPC",
    "China - Shanghai Composite": "000001.SS",
    "Hong Kong - HANG SENG INDEX": "^HSI",
    "Australia - S&P/ASX 200": "^AXJO",
    "India - Nifty 50": "^NSEI",
    "Canada - S&P/TSX Composite": "^GSPTSE",
    "Germany - DAX": "^GDAXI",
    "United Kingdom - FTSE 100": "^FTSE",
    "Japan - Nikkei 225": "^N225",
    "Mexico - IPC Mexico": "^MXX",
    "Brazil - Ibovespa": "^BVSP"
}

tickers = list(index_tickers.values())
data = yf.download(tickers=tickers, interval='1d')['Close']

#most index closed on the 1st of jan, so using ffill to add extra data
data_filled = data.ffill(axis=0)

# Reverse the index_tickers dictionary to map ticker to name
ticker_to_name = {v: k for k, v in index_tickers.items()}

[*********************100%***********************]  11 of 11 completed


In [101]:

# get data from 1st jan 2025 to 1st may 2025
get_report_by_date(data_filled,ticker_to_name,'2025-01-01','2025-05-01')

Period for indexes: 2025-01-01 to 2025-05-01 
 Rank                       Index YTD Return (%)
    1         Mexico - IPC Mexico         13.62%
    2               Germany - DAX         13.00%
    3           Brazil - Ibovespa         12.29%
    4 Hong Kong - HANG SENG INDEX         10.27%
    5   United Kingdom - FTSE 100          3.96%
    6            India - Nifty 50          2.49%
    7  Canada - S&P/TSX Composite          0.27%
    8     Australia - S&P/ASX 200         -0.17%
    9  China - Shanghai Composite         -2.17%
   10     United States - S&P 500         -4.72%
   11          Japan - Nikkei 225         -8.63%


Bonus question

In [102]:
# 3 years ago
today = date.today().strftime("%Y-%m-%d")
three_years_ago = (date.today() - timedelta(days=3*365)).strftime("%Y-%m-%d")
get_report_by_date(data_filled,ticker_to_name,three_years_ago,today)

Period for indexes: 2022-06-08 to 2025-06-07 
 Rank                       Index YTD Return (%)
    1               Germany - DAX         68.24%
    2            India - Nifty 50         52.87%
    3     United States - S&P 500         45.79%
    4          Japan - Nikkei 225         33.67%
    5  Canada - S&P/TSX Composite         27.11%
    6           Brazil - Ibovespa         25.59%
    7     Australia - S&P/ASX 200         19.58%
    8         Mexico - IPC Mexico         16.54%
    9   United Kingdom - FTSE 100         16.40%
   10 Hong Kong - HANG SENG INDEX          8.08%
   11  China - Shanghai Composite          3.72%


In [103]:
# 5 years ago
today = date.today().strftime("%Y-%m-%d")
three_years_ago = (date.today() - timedelta(days=5*365)).strftime("%Y-%m-%d")
get_report_by_date(data_filled,ticker_to_name,three_years_ago,today)

Period for indexes: 2020-06-08 to 2025-06-07 
 Rank                       Index YTD Return (%)
    1            India - Nifty 50        145.91%
    2               Germany - DAX         89.59%
    3     United States - S&P 500         85.63%
    4  Canada - S&P/TSX Composite         65.44%
    5          Japan - Nikkei 225         62.83%
    6         Mexico - IPC Mexico         45.32%
    7     Australia - S&P/ASX 200         41.96%
    8           Brazil - Ibovespa         39.38%
    9   United Kingdom - FTSE 100         36.54%
   10  China - Shanghai Composite         15.24%
   11 Hong Kong - HANG SENG INDEX         -3.97%


In [104]:
# 10 years ago
today = date.today().strftime("%Y-%m-%d")
three_years_ago = (date.today() - timedelta(days=10*365)).strftime("%Y-%m-%d")
get_report_by_date(data_filled,ticker_to_name,three_years_ago,today)

Period for indexes: 2015-06-10 to 2025-06-07 
 Rank                       Index YTD Return (%)
    1            India - Nifty 50        207.75%
    2     United States - S&P 500        185.03%
    3           Brazil - Ibovespa        152.62%
    4               Germany - DAX        115.74%
    5          Japan - Nikkei 225         88.27%
    6  Canada - S&P/TSX Composite         77.51%
    7     Australia - S&P/ASX 200         55.44%
    8         Mexico - IPC Mexico         30.25%
    9   United Kingdom - FTSE 100         29.39%
   10 Hong Kong - HANG SENG INDEX        -10.85%
   11  China - Shanghai Composite        -33.70%
